In [16]:
"""Task 0: Transfer Knowledge"""
import tensorflow.keras as K
import tensorflow as tf

(X_train, Y_train), (X_test, Y_test) = K.datasets.cifar10.load_data()


def preprocess_data(X, Y):
    """Pre-processes the data for the model
    Args:
        X: numpy.ndarray of shape (m, 32, 32, 3) containing the CIFAR 10 data,
            where m is the number of data points
        Y: numpy.ndarray of shape (m,) containing the CIFAR 10 labels for X
    Returns:
        X_p: preprocessed X
        Y_p: preprocessed Y
    """
    X_p = K.applications.efficientnet_v2.preprocess_input(X)
    Y_p = K.utils.to_categorical(Y, 10)
    return X_p, Y_p

In [17]:
MODEL_PATH = 'cifar10.h5'

print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print('X_test shape:', X_test.shape)
print('Y_test shape:', Y_test.shape)

X_train, Y_train = preprocess_data(X_train, Y_train)
X_test, Y_test = preprocess_data(X_test, Y_test)

inputs = K.Input(shape=(32, 32, 3))
upscale = K.layers.experimental.preprocessing.Resizing(244, 224)(inputs)
base_model = K.applications.EfficientNetV2B0(include_top=False,
                                             weights='imagenet',
                                             input_tensor=upscale,
                                             input_shape=(244, 224, 3))
base_model.trainable = False
out = base_model.output
out = K.layers.Flatten()(out)
out = K.layers.Dense(10, activation='softmax')(out)
fullmodel = K.models.Model(inputs=inputs, outputs=out)
fullmodel.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
fullmodel.fit(X_train, Y_train, batch_size=32, epochs=5,
              validation_data=(X_test, Y_test))
fullmodel.save(MODEL_PATH)

X_train shape: (50000, 32, 32, 3)
Y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 1)
Epoch 1/5
1563/1563 [==============================] - 1145s 729ms/step - loss: 2.2233 - accuracy: 0.8519 - val_loss: 2.0957 - val_accuracy: 0.8921
Epoch 2/5
1563/1563 [==============================] - 1219s 780ms/step - loss: 1.3089 - accuracy: 0.9238 - val_loss: 2.7791 - val_accuracy: 0.8910
Epoch 3/5
1563/1563 [==============================] - 1243s 795ms/step - loss: 1.0281 - accuracy: 0.9430 - val_loss: 2.8170 - val_accuracy: 0.9012
Epoch 4/5
1563/1563 [==============================] - 1137s 728ms/step - loss: 0.8901 - accuracy: 0.9537 - val_loss: 3.5423 - val_accuracy: 0.8976
Epoch 5/5
1563/1563 [==============================] - 1091s 698ms/step - loss: 0.7913 - accuracy: 0.9601 - val_loss: 3.1991 - val_accuracy: 0.9077


In [ ]:
"""Task 0: Main file"""

K.learning_phase = K.backend.learning_phase 

X_p, Y_p = preprocess_data(X_test, Y_test)

load_model = K.models.load_model('cifar10.h5')
load_model.evaluate(X_p, Y_p, batch_size=128, verbose=1)